In [1]:
def triangulacion(A, b, n):
    for k in range(n):
        for i in range(k+1, n):
            A[i][k] /= A[k][k]
            for j in range(k+1, n):
                A[i][j] -=  A[i][k] * A[k][j]
    return sustitucion(A, b, n)



def sustitucion(A, b, n):
    x = [0.0] * n
    y = [0.0] * n

    # Resolución Ly = b
    y[0] = b[0]
    for k in range(1, n):
        suma = 0.0


        
        for j in range(k):
            suma += A[k][j] * y[j]
        y[k] = b[k] - suma


    
    # Resolución Ux = y
    x[n-1] = y[n-1] / A[n-1][n-1]
    
    for i in range(n-2, -1, -1):
        suma = 0.0

        
        for j in range(i+1, n):
            suma += A[i][j] * x[j]
            
        x[i] = (y[i] - suma) / A[i][i]

    return x



Solución del sistema:
x[0] = -1.8750
x[1] = 0.9167
x[2] = -1.3333


In [7]:
def triangulacionParcial(A, b, n):
    for k in range(n):

        
        # Pivoteo parcial por columna
        max_row = max(range(k, n), key=lambda i: abs(A[i][k]))
        if A[max_row][k] == 0:
            raise ValueError("Matriz singular")


        
        # Intercambio de filas en A y b
        A[k], A[max_row] = A[max_row], A[k]
        b[k], b[max_row] = b[max_row], b[k]


        

        #igual que en triangulacion simple
        for i in range(k + 1, n):
            A[i][k] /= A[k][k]
            for j in range(k + 1, n):
                A[i][j] -= A[i][k] * A[k][j]



    
    return sustitucion(A, b, n)



In [8]:
def triangulacionParcialEscalada(A, b, n): #mejor metodo
    s = [max(abs(A[i][j]) for j in range(n)) for i in range(n)]  # factores de escala

    for k in range(n):
        # Buscar el índice con el mayor cociente relativo, de aca para 
        max_ratio = -1
        max_row = k
        
        for i in range(k, n):
            
            if s[i] == 0:
                raise ValueError("Fila con todos ceros en A")
                
            ratio = abs(A[i][k]) / s[i]
            
            if ratio > max_ratio:
                max_ratio = ratio
                max_row = i

        # Intercambio de filas 
        A[k], A[max_row] = A[max_row], A[k]
        b[k], b[max_row] = b[max_row], b[k]
        s[k], s[max_row] = s[max_row], s[k]
        #para aca se agrega nueva cosas

        
        
        #igual que en triangulacion simple
        for i in range(k + 1, n):
            A[i][k] /= A[k][k]
            for j in range(k + 1, n):
                A[i][j] -= A[i][k] * A[k][j]

    return sustitucion(A, b, n)


In [9]:



def jacobi(A, b, max_iter, eps):
    #mido la longuitud de la matriz y creo 2 una nueva y vieja
    n = len(A)
    x_old = [0.0] * n
    x_new = [0.0] * n

    for k in range(1, max_iter + 1):
        
        for i in range(n):
            
            suma = 0.0
            
            for j in range(n):
                
                if j != i: 
                    suma += A[i][j] * x_old[j]
                    
            x_new[i] = (b[i] - suma) / A[i][i]

        # Criterio de convergencia
        error = max(abs(x_new[i] - x_old[i]) for i in range(n))

        # Imprimir iteración y error
        print(f"Iteración {k}: error = {error:.6f}")

        if error < eps:
            return x_new

        x_old = x_new[:]

    return x_new


In [ ]:
def GaussSeidel(A, b, max_iter, eps):
    n = len(A)
    x_old = [0.0] * n #RESULTADO VIEJO
    x_new = [0.0] * n

    for k in range(1, max_iter + 1):
        for i in range(n):
            suma = 0.0
            
            for j in range(n):
                if j < i:
                    suma += A[i][j] * x_new[j]  # valores nuevos
                elif j > i:
                    suma += A[i][j] * x_old[j]  # valores viejos

            x_new[i] = (b[i] - suma) / A[i][i]

        error = max(abs(x_new[i] - x_old[i]) for i in range(n))
        
        print(f"Iteración {k}: error = {error:.6f}")

        if error < eps:
            return x_new

        x_old = x_new[:]

    return x_new


In [11]:
def SOR(A, b, max_iter, eps, omega):
    n = len(A)
    x_old = [0.0] * n
    x_new = [0.0] * n

    for k in range(1, max_iter + 1):
        for i in range(n):
            suma = 0.0
            # suma para j < i con valores nuevos
            for j in range(i):
                suma += A[i][j] * x_new[j]
            # suma para j > i con valores viejos
            for j in range(i+1, n):
                suma += A[i][j] * x_old[j]

            # Fórmula SOR:
            x_i_old = x_old[i]
            x_new[i] = (1 - omega) * x_i_old + (omega * (b[i] - suma)) / A[i][i]

        error = max(abs(x_new[i] - x_old[i]) for i in range(n))
        print(f"Iteración {k}: error = {error:.6f}")

        if error < eps:
            return x_new

        x_old = x_new[:]

    return x_new


In [15]:
# Ejemplo de uso
A = [
    [1.0, -1.0, -1.0, -1.0],
    [1.0, 0.0, 2.0,2.0],
    [1.0, 1.0, 2.0,-1.0],
    [-1.0, 3.0, 1.0,0.0]
]
b = [1.0,-1.0, 2.0,-1.0]
n = len(A)

x = triangulacion(A, b, n)

print("Solución del sistema:")
for i in range(n):
    print(f"x[{i}] = {x[i]:.4f}")


Solución del sistema:
x[0] = -0.0667
x[1] = -0.6000
x[2] = 0.7333
x[3] = -1.2000


In [ ]:
B = [
    [2.0, -1.0, 0.0],
    [1.0, 6.0, -2.0],
    [4.0, -3.0, 8.0]
    
]
c = [2.0,-4.0, 5.0]
n = len(B)

x = GaussSeidel(B, c, 88, 10e-6)

print("Solución del sistema:")
for i in range(n):
    print(f"x[{i}] = {x[i]:.4f}")

In [1]:
"""
/*import numpy as np
from scipy.sparse import lil_matrix
from scipy.sparse.linalg import norm
import os

def leer_matriz_dispersa(nombre_archivo, n):
    A = lil_matrix((n, n))
    with open(nombre_archivo, 'r') as archivo:
        for i, linea in enumerate(archivo):
            valores = [float(x) for x in linea.strip().split(',')]
            for j, valor in enumerate(valores):
                if valor != 0.0:
                    A[i, j] = valor
    return A

def leer_vector(nombre_archivo):
    with open(nombre_archivo, 'r') as archivo:
        return np.array([float(x) for x in archivo.readline().strip().split(',')])

def gauss_seidel_disperso(A, b, x0, max_iter=1000, tol=1e-6):
    n = A.shape[0]
    x = x0.copy()

    for k in range(max_iter):
        x_old = x.copy()
        for i in range(n):
            suma = 0.0
            for j in range(n):
                if j != i and A[i, j] != 0:
                    suma += A[i, j] * x[j]
            x[i] = (b[i] - suma) / A[i, i]

        error = np.linalg.norm(x - x_old, np.inf)
        if error < tol:
            break

    return x

# Definir ruta base a tus archivos
ruta_base = r"C:\Users\facu\Downloads\facultad\MN1\practica"
ruta_A = os.path.join(ruta_base, "A.txt")
ruta_b = os.path.join(ruta_base, "b.txt")
ruta_x = os.path.join(ruta_base, "x.txt")

# Primero necesitás saber n, la dimensión del sistema
# Asumimos que A.txt tiene n filas
with open(ruta_A, 'r') as f:
    n = sum(1 for _ in f)

# Leemos la matriz dispersa y los vectores
A = leer_matriz_dispersa(ruta_A, n)
b = leer_vector(ruta_b)
x0 = leer_vector(ruta_x)

# Ejecutamos Gauss-Seidel para matriz dispersa
x_sol = gauss_seidel_disperso(A, b, x0, max_iter=1000, tol=1e-6)

#print("Solu#ción aproximada:")
print(x_sol)
"""

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 1156-1157: truncated \UXXXXXXXX escape (2708768769.py, line 1)